# Stats

This notebook loads some data, reports some simple descriptive statistics (means, standard deviationsm etc) and show a number of useful plots (scatter plots, histograms, time series plots).

Most of the descriptive stats use built-in Julia commands. The plots reply on the Plots package and the pdf and quantiles are from the Distributions package (see https://github.com/JuliaStats/Distributions.jl).

## Load Packages

In [1]:
using Distributions      #distributions, random numbers, etc.
                         
include("printmat.jl")   #just a function for prettier matrix printing

println4Ps (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Load Data from Text File

The following is a portion of MyData.csv:

```
date,Mkt-RF,RF,SmallGrowth
197901,4.18,0.77,10.96
197902,-3.41,0.73,-2.09
197903,5.75,0.81,11.71
197904,0.05,0.8,3.27
```

In [3]:
xx   = readdlm("Data/MyData.csv",',',header=true)  #reading the csv file    
x    = xx[1]                                #xx[1] contains the data

println("Column headers: ",xx[2])           #xx[2] contains the headers
println("\nfirst four lines of x:") 
printmat(x[1:4,:])

Column headers: AbstractString["date" "Mkt-RF" "RF" "SmallGrowth"]

first four lines of x:
197901.000     4.180     0.770    10.960
197902.000    -3.410     0.730    -2.090
197903.000     5.750     0.810    11.710
197904.000     0.050     0.800     3.270



### Creating Variables

In [4]:
ym  = x[:,1]                    #yearmonth, like 200712
Rme = x[:,2]                    #picking out the second column
Rf  = x[:,3]                   
R   = x[:,4] +                  # + 0.0 to illustrate that commands continue on the next line
      0.0                     
Re  = R - Rf                    #do R .- Rf if R has several columns (and Rf only one)

println("first 4 obs of Rme and Re")
printmat([Rme[1:4,:] Re[1:4,:]])

first 4 obs of Rme and Re
     4.180    10.190
    -3.410    -2.820
     5.750    10.900
     0.050     2.470



# Some Descriptive Statistics

## Means and Standard Deviations

The next few cells estimate means, standard deviations, covarances and correlations of the variables Rme (US equity market excess return) and Re (excess returns for a segment of the market, small growth firms). 

In [5]:
μ = mean([Rme Re],1)    #,1 to calculate average along a column, gives a row vector
σ = std([Rme Re],1)     #do \sigma[Tab] to get σ

println("            Rme       Re")
printlnPs("means: ",μ)  #for more stat functions, see the package StatsBase.jl
printlnPs("std:   ",σ)

            Rme       Re
means:      0.602     0.303
std:        4.604     8.572


## Covariances and Correlations

In [6]:
println("\n","cov([Rme Re]): ")          
printmat(cov([Rme Re]))

println("\n","cor([Rme Re]): ")          
printmat(cor([Rme Re]))


cov([Rme Re]): 
    21.197    28.426
    28.426    73.475


cor([Rme Re]): 
     1.000     0.720
     0.720     1.000



## OLS

A simple linear regression
$
y = b x + u
$,
where $x=[1,R^e_m]$.

Clearly, the first element of b is the intercept and the second element is the slope coefficient.

The GLM package (not used here) has powerful regression methods. See https://github.com/JuliaStats/GLM.jl.

In [7]:
c   = ones(size(Rme,1))         #a vector with ones, no. rows from variable, here ones(Rme) works too
x   = [c Rme]                   #x is a Tx2 matrix
y   = copy(Re)                  #to get standard OLS notation, copy to get an independent copy

b   = inv(x'x)*x'y              #OLS according to a textbook
b2  = x\y                       #also OLS, quicker and numerically more stable
u   = y - x*b                   #OLS residuals
R2a = 1 - var(u)/var(y)         #R2, but that name is already taken

println("OLS coefficients, regressing Re on constant and Rme, different calculations")
printmat([b b2])                
printlnPs("R2: ",R2a) 
printlnPs("no. of observations: ",size(Re,1))

OLS coefficients, regressing Re on constant and Rme, different calculations
    -0.504    -0.504
     1.341     1.341

R2:      0.519
no. of observations:        388


# Drawing Random Numbers and Finding Critical Values

## Random Numbers: Independent Variables

In [8]:
T = 100
x = randn(T,2)    #T x 2 matrix, normal distribution

println("\n","mean and std of random draws: ")
mu    = mean(x,1)                 
sigma = std(x,1)
printmat([mu;sigma])

println("covariance and correlation matrices:")
printmat(cov(x))
printmat(cor(x))


mean and std of random draws: 
    -0.107    -0.004
     0.991     0.974

covariance and correlation matrices:
     0.981     0.208
     0.208     0.949

     1.000     0.216
     0.216     1.000



## Random Numbers: Correlated Variables

In [9]:
mu = [-1 10]                 #means of two series
V  = [1   0.5;               #covariance matrix of two series
      0.5 2  ]

T = 100
x = mu .+ randn(T,2)*chol(V)    #T x 2 matrix, multivariate normal distribution

println("\n","mean and std of random draws: ")
mu    = mean(x,1)                 
sigma = std(x,1)
printmat([mu;sigma])

println("covariance and correlation matrices:")
printmat(cov(x))
printmat(cor(x))


mean and std of random draws: 
    -0.935     9.939
     1.027     1.527

covariance and correlation matrices:
     1.055     0.652
     0.652     2.332

     1.000     0.415
     0.415     1.000



## Quantiles ("critical values") of Distributions

In [10]:
N05     = quantile(Normal(0,1),0.05)            #from the Distributions package
Chisq05 = quantile(Chisq(5),0.95)

println("\n","5th percentile of N(0,1) and 95th of Chisquare(5)")      #lots of statistics functions
printmat([N05 Chisq05])


5th percentile of N(0,1) and 95th of Chisquare(5)
    -1.645    11.070



# Statistical Plots

In [11]:
YearFrac = floor(ym/100) + (mod(ym,100)-1)/12    #year + (month-1)/12, simple but works

plot3a = plot(YearFrac,Rme,color=:blue,legend=false)
plot!(xlims=(1978,2012),ylims=(-25,25))
title!("Time series plot: monthly US equity market excess return")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 Time series plot: monthly US equity market excess return 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.0633,173.939 63.4117,228.826 64.7602,162.586 66.1087,203.805 67.4572,219.931 68.8057,176.109 70.1542,198.888 71.5027,162.948 72.8511,209.156 74.1996,263.03 
 75.5481,165.334 76.8966,190.644 78.2451,162.514 79.5936,209.879 80.942,299.838 82.2905,175.458 83.639,166.563 84.9875,181.315 86.336,157.813 87.6845,191.729 
 89.0329,188.258 90.3814,196.501 91.7299,135.179 93.0784,238.516 94.4269,240.685 95.7754,200.696 97.1239,179.508 98.4723,220.148 99.8208,202.648 101.169,221.305 
 102.518,215.375 103.866,254.136 105.215,259.27 106.563,169.384 107.912,178.784 109.26,230.778 110.609,228.898 111.957,247.772 113.306,218.557 114.654,181.026 
 116.003,232.225 117.351,228.392 118.7,226.584 120.048,123.609 121.397,195.706 122.745,122.669 124.094,171.191 125.442,198.526 126.791,178.857 128.139,186.811 
 129.487,183.629 130.836,155.644 132.184,199.611 133.533,181.677 134.881,232.369 136.23,207.132 137.578,198.02 138.927,229.91 140.275,187.824 141.624,217.039 
 142.972,219.063 144.321,237.576 145.669,199.756 147.018,208.216 148.366,247.627 149.715,192.669 151.063,224.993 152.412,128.671 153.76,210.096 155.109,211.47 
 156.457,217.183 157.806,191.656 159.154,146.894 160.503,196.14 161.851,209.879 163.2,210.964 164.548,168.588 165.897,195.778 167.245,208.867 168.594,211.615 
 169.942,237.287 171.29,176.76 172.639,158.536 173.987,177.7 175.336,201.129 176.684,155.572 178.033,169.528 179.381,213.64 180.73,170.975 182.078,197.658 
 183.427,251.099 184.775,159.621 186.124,264.549 187.472,171.842 188.821,196.067 190.169,226.801 191.518,114.28 192.866,172.638 194.215,190.427 195.563,219.642 
 196.912,203.227 198.26,176.036 199.609,175.53 200.957,180.737 202.306,222.462 203.654,371.501 205.003,258.981 206.351,156.15 207.7,173.795 209.048,170.107 
 210.397,219.353 211.745,199.539 213.093,207.565 214.442,170.468 215.79,213.134 217.139,228.681 218.487,181.749 219.836,195.851 221.184,220.148 222.533,193.464 
 223.881,160.344 225.23,220.437 226.578,193.464 227.927,174.156 229.275,181.46 230.624,212.844 231.972,153.474 233.321,193.536 234.669,209.952 236.018,230.272 
 237.366,196.284 238.715,195.344 240.063,258.981 241.412,197.514 242.76,191.367 244.109,229.621 245.457,144.797 246.806,211.76 248.154,215.882 249.503,275.396 
 250.851,247.41 252.2,218.123 253.548,160.778 254.896,187.173 256.245,172.421 257.593,152.824 258.942,186.45 260.29,205.613 261.639,178.134 262.987,239.022 
 264.336,173.867 265.684,188.113 267.033,215.448 268.381,194.332 269.73,233.96 271.078,129.683 272.427,207.493 273.775,196.501 275.124,223.764 276.472,196.791 
 277.821,201.563 279.169,220.437 280.518,177.555 281.866,221.088 283.215,197.08 284.563,197.875 285.912,176.76 287.26,193.32 288.609,196.718 289.957,201.853 
 291.306,187.824 292.654,224.27 294.003,184.353 295.351,202.07 296.7,206.481 298.048,177.41 299.396,205.613 300.745,192.669 302.093,218.702 303.442,191.729 
 304.79,183.196 306.139,223.185 307.487,239.239 308.836,199.249 310.184,199.683 311.533,226.584 312.881,184.063 314.23,176.036 315.578,220.148 316.927,196.429 
 318.275,233.743 319.624,198.237 320.972,192.452 322.321,178.423 323.669,187.968 325.018,189.27 326.366,183.485 327.715,185.003 329.063,177.917 330.412,200.84 
 331.76,180.954 333.109,215.737 334.457,176.326 335.806,196.718 337.154,186.956 338.503,195.2 339.851,199.105 341.199,189.053 342.548,187.824 343.896,213.061 
 345.245,246.326 346.593,183.629 347.942,169.022 349.29,197.297 350.639,159.694 351.987,215.737 353.336,168.733 354.684,207.782 356.033,239.745 357.381,176.615 
 358.73,155.933 360.078,174.952 361.427,151.956 362.775,233.382 364.124,165.045 365.472,232.08 366.821,

In [12]:
plot3b = scatter(Rme,Re,color=:blue,legend=false)
plot!([-40;60],[-40;60],color=:black)
plot!(xlims=(-40,40),ylims=(-40,60))
title!("Scatter plot: two monthly return series (and 45 degree line)")
xlabel!("Market excess return, %")
ylabel!("Excess returns on small growth stocks, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -40 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 -40 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 Scatter plot: two monthly return series (and 45 degree line) 
 
 
 Market excess return, % 
 
 
 Excess returns on small growth stocks, %

In [13]:
xGrid = -25:0.1:15
pdfX   = pdf(Normal(mean(Rme),std(Rme)),xGrid) #the N(μ,σ) pdf
                                        #"Distributions" wants σ, not σ^2

histogram(Rme,bins = -25:1:15,normalized=true,label="histogram")
plot!(xGrid,pdfX,linewidth=3,label="fitted N()")
title!("Histogram: monthly US equity market excess return")
xlabel!("Market excess return, %")
ylabel!("Number of months")

INFO: binning = -25:1:15


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 Histogram: monthly US equity market excess return 
 
 
 Market excess return, % 
 
 
 Number of months 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 70.0899,369.674 71.3666,369.674 72.6432,369.674 73.9198,369.674 75.1965,369.674 76.4731,369.674 77.7497,369.674 79.0264,369.674 80.303,369.674 81.5796,369.674 
 82.8563,369.674 84.1329,369.674 85.4095,369.674 86.6862,369.674 87.9628,369.674 89.2394,369.674 90.5161,369.674 91.7927,369.674 93.0693,369.674 94.346,369.674 
 95.6226,369.674 96.8993,369.674 98.1759,369.673 99.4525,369.673 100.729,369.673 102.006,369.673 103.282,369.673 104.559,369.673 105.836,369.673 107.112,369.673 
 108.389,369.673 109.666,369.672 110.942,369.672 112.219,369.672 113.495,369.672 114.772,369.672 116.049,369.671 117.325,369.671 118.602,369.671 119.879,369.67 
 121.155,369.67 122.432,369.669 123.709,369.669 124.985,369.668 126.262,369.668 127.538,369.667 128.815,369.666 130.092,369.665 131.368,369.665 132.645,369.664 
 133.922,369.663 135.198,369.661 136.475,369.66 137.752,369.659 139.028,369.657 140.305,369.655 141.581,369.654 142.858,369.651 144.135,369.649 145.411,369.647 
 146.688,369.644 147.965,369.641 149.241,369.638 150.518,369.635 151.794,369.631 153.071,369.627 154.348,369.622 155.624,369.618 156.901,369.612 158.178,369.607 
 159.454,369.6 160.731,369.594 162.008,369.586 163.284,369.579 164.561,369.57 165.837,369.561 167.114,369.551 168.391,369.54 169.667,369.528 170.944,369.515 
 172.221,369.501 173.497,369.486 174.774,369.47 176.051,369.453 177.327,369.434 178.604,369.414 179.88,369.392 181.157,369.368 182.434,369.343 183.71,369.316 
 184.987,369.287 186.264,369.255 187.54,369.221 188.817,369.185 190.094,369.146 191.37,369.104 192.647,369.059 193.923,369.011 195.2,368.96 196.477,368.905 
 197.753,368.846 199.03,368.783 200.307,368.715 201.583,368.643 202.86,368.566 204.136,368.484 205.413,368.397 206.69,368.303 207.966,368.204 209.243,368.098 
 210.52,367.985 211.796,367.865 213.073,367.737 214.35,367.602 215.626,367.458 216.903,367.305 218.179,367.142 219.456,366.97 220.733,366.787 222.009,366.594 
 223.286,366.389 224.563,366.172 225.839,365.943 227.116,365.7 228.393,365.444 229.669,365.173 230.946,364.887 232.222,364.585 233.499,364.267 234.776,363.932 
 236.052,363.579 237.329,363.207 238.606,362.816 239.882,362.404 241.159,361.971 242.435,361.517 243.712,361.04 244.989,360.539 246.265,360.014 247.542,359.463 
 248.819,358.886 250.095,358.282 251.372,357.649 252.649,356.988 253.925,356.296 255.202,355.573 256.478,354.819 257.755,354.031 259.032,353.209 260.308,352.352 
 261.585,351.46 262.862,350.53 264.138,349.562 265.415,348.556 266.692,347.509 267.968,346.422 269.245,345.293 270.521,344.121 271.798,342.905 273.075,341.645 
 274.351,340.339 275.628,338.987 276.905,337.587 278.181,336.14 279.458,334.644 280.734,333.098 282.011,331.503 283.288,329.856 284.564,328.158 285.841,326.408 
 287.118,324.606 288.394,322.75 289.671,320.841 290.948,318.879 292.224,316.862 293.501,314.792 294.777,312.667 296.054,310.488 297.331,308.254 298.607,305.966 
 299.884,303.625 301.161,301.229 302.437,298.78 303.714,296.278 304.991,293.724 306.267,291.117 307.544,288.46 308.82,285.752 310.097,282.995 311.374,280.19 
 312.65,277.337 313.927,274.438 315.204,271.495 316.48,268.508 317.757,265.48 319.033,262.412 320.31,259.305 321.587,256.162 322.863,252.985 324.14,249.775 
 325.417,246.535 326.693,243.267 327.97,239.974 329.247,236.658 330.523,233.321 331.8,229.966 333.076,226.596 334.353,223.214 335.63,219.823 336.906,216.426 
 338.183,213.026 339.46,209.625 340.

In [14]:
println("\n","end of program")


end of program
